### image recognization

In [77]:
import numpy as np
from glob import glob
import cv2
import matplotlib.pyplot as plt
import pdb

np.random.seed(0)
# xs = np.array([[0,0],[0,1],[1,0],[1,1]],dtype=np.float32)
# ts = np.array([[0],[1],[1],[0]],dtype=np.float32)

In [78]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [89]:
class FullyConnectedLayer():
    def __init__(self, in_n, out_n, use_bias=True, activation=None):
        self.w = np.random.normal(0, 1, [in_n, out_n])
        if use_bias:
            self.b = np.random.normal(0, 1, [out_n])
        else:
            self.b = None
        if activation is not None:
            self.activation = activation
        else:
            self.activation = None

    def set_lr(self, lr=0.1):
        self.lr = lr

    def forward(self, feature_in):
        self.f_in = feature_in
        x = np.dot(feature_in, self.w)
        
        if self.b is not None:
            x += self.b
            
        if self.activation is not None:
            x = self.activation(x)
        self.ys = x
        
        return x

    
    def backward(self, w_pro, grad_pro):
        grad_u = np.dot(grad_pro, w_pro.T)
        if self.activation is sigmoid:
            grad_u *= (self.ys * (1 - self.ys))
        grad_w = np.dot(self.f_in.T, grad_u)
        self.w -= self.lr * grad_w

        if self.b is not None:
            grad_b = np.dot(np.ones([grad_u.shape[0]]), grad_u)
            self.b -= self.lr * grad_b

        return grad_u

In [97]:
class Model():
    def __init__(self, *args, lr=0.1):
        self.layers = args
        for l in self.layers:
            l.set_lr(lr=lr)

    def forward(self, x):
        for l in self.layers:
            x = l.forward(x)
        self.ys = x
        
        return x

    def backward(self, t):
        En = -(t - self.ys) * self.ys * (1 - self.ys)
        grad_pro = En
        w_pro = np.eye(En.shape[-1])
        
        for i, l in enumerate(self.layers[::-1]):
            grad_pro = l.backward(w_pro=w_pro, grad_pro=grad_pro)
            w_pro = l.w


    def loss(self, t):
        Loss = np.sum((self.ys - t) ** 2) / 2 / t.shape[0]
        return Loss   

In [98]:
num_classes = 2
img_height, img_width = 64, 64

CLS = ['akahara', 'madara']

In [99]:
# get train data
def data_load(path, hf=False, vf=False, rot=None):
    xs = []
    ts = []
    paths = []
    
    for dir_path in glob(path + '/*'):
        for path in glob(dir_path + '/*'):
            x = cv2.imread(path)
            x = cv2.resize(x, (img_width, img_height)).astype(np.float32)
            x /= 255.
            x = x[..., ::-1]
            xs.append(x)

            for i, cls in enumerate(CLS):
                if cls in path:
                    t = i
            
            ts.append(t)

            paths.append(path)

            if hf:
                xs.append(x[:, ::-1])
                ts.append(t)
                paths.append(path)

            if vf:
                xs.append(x[::-1])
                ts.append(t)
                paths.append(path)

            if hf and vf:
                xs.append(x[::-1, ::-1])
                ts.append(t)
                paths.append(path)

            if rot is not None:
                angle = rot
                scale = 1

                # show
                a_num = 360 // rot
                w_num = np.ceil(np.sqrt(a_num))
                h_num = np.ceil(a_num / w_num)
                count = 1
                
                while angle < 360:
                    _h, _w, _c = x.shape
                    max_side = max(_h, _w)
                    tmp = np.zeros((max_side, max_side, _c))
                    tx = int((max_side - _w) / 2)
                    ty = int((max_side - _h) / 2)
                    tmp[ty: ty+_h, tx: tx+_w] = x.copy()
                    M = cv2.getRotationMatrix2D((max_side/2, max_side/2), angle, scale)
                    _x = cv2.warpAffine(tmp, M, (max_side, max_side))
                    _x = _x[tx:tx+_w, ty:ty+_h]
                    xs.append(x)
                    ts.append(t)
                    paths.append(path)
                    angle += rot

    ts = [[t] for t in ts]
                    
    xs = np.array(xs, dtype=np.float32)
    ts = np.array(ts, dtype=np.int)
    
    xs = xs.transpose(0,3,1,2)

    return xs, ts, paths

In [100]:
model = Model(FullyConnectedLayer(in_n=img_height * img_width * 3, out_n=64, activation=sigmoid),
              FullyConnectedLayer(in_n=64, out_n=32, activation=sigmoid),
              FullyConnectedLayer(in_n=32, out_n=1, activation=sigmoid), lr=0.1)

In [101]:
xs, ts, paths = data_load("Dataset/train/images/", hf=True, vf=True, rot=1)

In [102]:
mb = 64
mbi = 0
train_ind = np.arange(len(xs))
np.random.shuffle(train_ind)

In [103]:
for ite in range(1000):
    if mbi + mb > len(xs):
        mb_ind = train_ind[mbi:]
        np.random.shuffle(train_ind)
        mb_ind = np.hstack((mb_ind, train_ind[:(mb-(len(xs)-mbi))]))
        mbi = mb - (len(xs) - mbi)
    else:
        mb_ind = train_ind[mbi: mbi+mb]
        mbi += mb

    x = xs[mb_ind]
    t = ts[mb_ind]

    x = x.reshape(mb, -1)

    model.forward(x)
    model.backward(t)
    loss = model.loss(t)

    if ite % 50 == 0:
        print("ite:", ite+1, "Loss >>", loss)

ite: 1 Loss >> 0.21352740372282433
ite: 51 Loss >> 0.009437671088643011
ite: 101 Loss >> 0.005869995744625613
ite: 151 Loss >> 0.004181180016093243
ite: 201 Loss >> 0.0030237996730212184
ite: 251 Loss >> 0.009818056301096513
ite: 301 Loss >> 0.008624463963603394
ite: 351 Loss >> 0.0017459336562101223
ite: 401 Loss >> 0.0031621349939443787
ite: 451 Loss >> 0.0015597370422339187
ite: 501 Loss >> 0.0014936561981282737
ite: 551 Loss >> 0.0013149378987216162
ite: 601 Loss >> 0.0017548737235955
ite: 651 Loss >> 0.008893150163254178
ite: 701 Loss >> 0.0010882634046149725
ite: 751 Loss >> 0.0011825437126263651
ite: 801 Loss >> 0.001204520596524986
ite: 851 Loss >> 0.001082104480260695
ite: 901 Loss >> 0.0010884577946439235
ite: 951 Loss >> 0.0009649387007242976


In [104]:
# test
xs, ts, paths = data_load("Dataset/test/images/")

for i in range(len(xs)):
    x = xs[i]
    x = x.reshape(1, -1)
    out = model.forward(x)
    print("in >>", paths[i], ", out >>", out)

in >> Dataset/test/images/akahara/akahara_0009.jpg , out >> [[0.42357736]]
in >> Dataset/test/images/akahara/akahara_0010.jpg , out >> [[0.18002142]]
in >> Dataset/test/images/akahara/akahara_0011.jpg , out >> [[0.36873427]]
in >> Dataset/test/images/madara/madara_0010.jpg , out >> [[0.22497696]]
in >> Dataset/test/images/madara/madara_0009.jpg , out >> [[0.12101534]]
in >> Dataset/test/images/madara/madara_0011.jpg , out >> [[0.93109881]]
